In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import os 

# Imports added from modifying the original code
import random
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, Column, join
from hetdex_tools.get_spec import get_spectra

2022-11-11 07:14:11.785600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-11 07:14:11.951060: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-11 07:14:11.951089: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-11 07:14:11.992437: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-11 07:14:12.801228: W tensorflow/stream_executor/platform/de

Keras is a deep learning API written in Python, running on top of ther machine learning platform TensorFlow. Keras was developed with a focus on enabling fast experimentation.

So Keras is a high-level neural network library that runs on top of TensorFlow. Keras is more user friendly because it's built in Python.

In [2]:
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import History 

Comments and notes by me (Nick D) :) just to help me understand what's going on.

Callbacks: A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc)

Callbacks can be used to:
   * Monitor metrics by writing TensorBoard logs
   * Periodically save model to disk drive
   * Do early stopping
   * Get internal states and stats during training
   * and more!

In [3]:
# This is creating a 'History' callback object. This object keeps track of the accuracy, loss and other training metrics
# for each epoch in the memory.
# Not 100% sure, but it seems like this is what allows the code to make plots later on.
history = History()

In [4]:
# Not entirely sure what the number means. But I'm assuming it's purely a naming convention and does not
# mean anything specific.
training_index = "31_cut"

### Opening HDR3 Catalog and converting it into a Df

In [5]:
# Opening HDR3 detections catalog and converting it into a pandas DF
# HDR3 is detections HETDEX found
# Using HDR3 NEP and not H20 NEP because we can make SN cuts!
HDR_source_cat = fits.open('/home/jovyan/Hobby-Eberly-Telesco/hdr3/catalogs/source_catalog_3.0.1.fits', memmap = True)
HDR3_data = HDR_source_cat[1].data
HDR3_DF = pd.DataFrame(HDR3_data, columns=HDR3_data.columns.names)

# Columns we will then take from the entire data set (it was huge so we needed to determine what we wanted to look at specifically).
# As the name suggests, these are the ones that are useful to us!
useful_hdr3_cols = ['source_id', 'detectid',  'selected_det', 'ra_mean', 'dec_mean', 'fwhm', 'shotid', 'field',  'ra', 'dec', 'wave', 'wave_err', 'flux', 'flux_err', 'sn', 'sn_err', 'chi2', 'chi2_err',
'linewidth', 'linewidth_err', 'plya_classification', 'z_hetdex', 'z_hetdex_conf', 'combined_plae']

# From the original DFs, taking the useful columns
reduced_hdr3_df = HDR3_DF.loc[:, useful_hdr3_cols]

### Filtering data

In [6]:
# Removing data from before 2017 because it isn't good (not useful to us)
# No need to do this for H20 NEP
removed_bad_shots_hdr3_df = reduced_hdr3_df[reduced_hdr3_df.shotid.values >= 20180000000]

# This will give high confidence detections. Something we would want to do also. What is sn threshold that Valentina's code is having trouble with.
# Reason why, we want high-confidence Lya. If we are very confident sn and another filter, then that's what we consider high-conf lya.
# Once noise and high-confidence sample. We can start exanping on valentina's code and do our own stuff
hdr3_signal_to_noise_interval = removed_bad_shots_hdr3_df[removed_bad_shots_hdr3_df['sn'] > 6]


# Now I just take the sources stricly in the NEP field
hdr3_nep = hdr3_signal_to_noise_interval[hdr3_signal_to_noise_interval['field'] == 'nep']

Making skycoord object for hdr3 ra and dec

In [7]:
hdr3_skycoords = SkyCoord(hdr3_nep['ra'] * u.deg, hdr3_nep['dec'] * u.deg)

# QUESTIONS:
1. Would I have to check for duplicates before extracting? [x]
    - Repeated observations in same spot is okay, exactly same detection not.
        - Use wavelength to check, if its within a couple angstroms of each other.
        - Could use detectID as well.

2. Do I care about spec_err for now? For now, no. [x]

3. How big should my random sample be? []
- 1000

4. Should I check for Nan values or can I trust get_spectra wouldn't give me something like that? There's code in Valentina's files to check for Nan but they don't run well with my sample [x]
- Good sanity check to do.

5. My approach is to basically do exactly everything she did, but with my sample. Is that the right way to go? Or should I edit some stuff along the way. Like do I make cuts like she did? [x]


# To DO:

1. Fix the shape of how my samples are. Right now instead of it being like 100 rows and a lot of columns it's just (100 rows) and then when you index it's lots of columns. Strange!

    - LOOK INTO numpy reshape!

    - Once fixed ^ move forward!
    
    
   - numpy isnan or another numpy nan function!

### Extracts a random sample of sources in NEP. TAKES A LONG TIME. SKIP IF DONT HAVE TIME (have a saved version below)

In [ ]:
random_nep_LS = []

# make it 1000
samples_amount = 100

for i in range(samples_amount):
    #pick a random coordinate from the skycoord object
    random_coord = hdr3_skycoords[random.randint(0, hdr3_skycoords.size)]
    # extract spectra at this random coordinate and append to list, can't append to np array because of 
    # data type issues will convert to array after loop!
    extraction = get_spectra(random_coord)
    random_nep_LS.append(np.squeeze(extraction['spec'].value))
    
random_nep_spectra = np.array(random_nep_LS)

In [ ]:
#np.save(f"random_nep_sample_{samples_amount}", random_nep_spectra)

Current problem: working on getting the array to a normal shape!

FIGURED IT OUT! Just need to use np.squeeze.

In [23]:
# Loading in spectra samples
star_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/star_spec_reduced.npy") 
agn_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/all_agn_no_duplicates.npy")
lowz_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/lowz_no_duplicates.npy")
gal_spec = np.load("/home/jovyan/work/stampede2/Nick_Capstone_Project/Valentina_samples/gal_sample.npy")

In [25]:
lowz_spec.shape

(1046, 1036)

In [19]:
star_spec.shape

(48580, 1036)

In [10]:
# Loading in spectra sample
# The allow pickle part uses the pickle module which is not secure against maliciously constructed data
# Don't allow pickle if you don't trust the source, I do so it doesn't matter to me
# I have to do the allow_pickle = True for the code to not give me an error anyway xD
NEP_spec = np.load("random_nep_sample_100.npy", allow_pickle = True)

In [20]:
agn_spec

array([[1.e-05, 1.e-05, 1.e-05, ..., 1.e-05, 1.e-05, 1.e-05],
       [1.e-05, 1.e-05, 1.e-05, ..., 1.e-05, 1.e-05, 1.e-05],
       [1.e-05, 1.e-05, 1.e-05, ..., 1.e-05, 1.e-05, 1.e-05],
       ...,
       [1.e-05, 1.e-05, 1.e-05, ..., 1.e-05, 1.e-05, 1.e-05],
       [1.e-05, 1.e-05, 1.e-05, ..., 1.e-05, 1.e-05, 1.e-05],
       [1.e-05, 1.e-05, 1.e-05, ..., 1.e-05, 1.e-05, 1.e-05]])

In [12]:
NEP_spec

array([array([ 9.07855501,  9.07913544,  9.07970425, ..., 14.32231376,
              14.32247893, 14.32264229])                              ,
       array([0.34457016, 0.34474193, 0.3449128 , ..., 0.62208756, 0.62205755,
              0.62202781])                                                    ,
       array([[0.96457583, 0.96396224, 0.9633459 , ..., 1.10606205, 1.10609389,
               1.10612532],
              [5.81014892, 5.80936567, 5.80857551, ..., 5.12000739, 5.1198799 ,
               5.11975362]])                                                   ,
       array([[80.51525665, 80.31588671, 80.04700063, ..., 72.97852403,
               72.96503124, 72.95167888],
              [49.39000522, 49.41100746, 49.43189237, ..., 62.21856434,
               62.22131635, 62.22403898]])                             ,
       array([[-0.10021422, -0.10039396, -0.10057328, ...,  0.41514248,
                0.41512052,  0.41509877],
              [ 0.29053362,  0.29008124,  0.28962848, ..

In [21]:
agn_spec.shape

(555, 1036)

In [27]:
NEP_spec.shape

(100,)

In [ ]:
# This function checks for nan values.
# Interestingly enough, when you do nan != nan, you get True. Becuase NaN is unequal with anything.
# But with numbers, you'll get false.
def isNaN(num):
    return num != num

In [ ]:
# If there are any values in the imports that are 'nan' (Not a Number) we remove them.
# Go through each value in matrix and check if NaN
def remove_nan(file):
    for i in range(0,len(file)):
        for j in range(0,len(file[i])):
            if isNaN(file[i][j]):

                # if value is nan, make the value equal to 0.0001
                # I'm assuming this was to avoid errors
                file[i][j]=0.00001

remove_nan(star_spec)
remove_nan(agn_spec)
remove_nan(lowz_spec)
remove_nan(gal_spec)
# Adding NEP files to the remove NaN function calls
#remove_nan(NEP_spec)

In [ ]:
# random.seed(2) will make the random numbers predictable, so random numbers, but the same random numbers from the same set/seed
# so I'll get the same random numbers everytime, until I change the seed
np.random.seed(2)
# random.shuffle will shuffle the values in an array or list. Like shuffling a deck of cards
np.random.shuffle(star_spec)

Tying to split between testing and training belo. scikit learning has something to make randomize data set and training sets.

In [ ]:
# concatenate just joins arrays
all_images = np.concatenate((star_spec[::90],agn_spec,lowz_spec,gal_spec))
# making a training set
train_images = np.concatenate((star_spec[::90][:433],agn_spec[:444],lowz_spec[:837],gal_spec[:254]))
#train_images = agn_spec[:433]
#np.random.shuffle(train_images)

In [ ]:
# not sure what these val_images are, it seems to be the opposite half of the train_images.
val_images = np.concatenate((star_spec[::90][433:],agn_spec[444:],lowz_spec[837:],gal_spec[254:]))
#val_images = agn_spec[433:]
#np.random.shuffle(val_images)

In [ ]:
# This function will normalize the data.
# The idea is, there is such a large range of sources so it may be simpler for the algorithm to recognize sources
# if they are between 0 and 1.
# Dividing by the largest number preserves the information but scales it down.
# If we don't do this the sources could go from a very large number to a small one, which makes it harder for the algorithm
def Norm(data):
    new_data = []
    for spec in data:
        #print('spec_before', spec)
        spec = spec/spec.max()
        #print('spec_after', spec)
        #print('spec.max', spec.max())
        new_data.append(spec)
    return np.array(new_data)

In [ ]:
# Normalizing both data sets
train_images = Norm(train_images)
val_images = Norm(val_images)

In [ ]:
# Making cuts of the image sets and then normalizing them.
train_cut = train_images[:,88:1002]
val_cut = val_images[:,88:1002]
train_cut = Norm(train_cut)
val_cut = Norm(val_cut)

In [ ]:
latent_dim = 30
act = 'sigmoid'

In [ ]:
class Autoencoder(Model):
def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      #layers.InputLayer(input_shape=(1036)),
      layers.Dense(436, activation=act,name="e1"),
      #layers.Dropout(0.3),
      #layers.Dense(136, activation=act, name ='e2'),
      #BatchNormalization(),
      layers.Dropout(0.3),
      layers.Dense(latent_dim, activation=None, name = "e3"),
    ])
    self.decoder = tf.keras.Sequential([
      #layers.InputLayer(input_shape=(latent_dim,)),
      #layers.Dense(136, activation=act,name = "d1"),
      layers.Dense(436, activation=act,name = "d2"),
      #BatchNormalization(),
      layers.Dense(914, activation=None,name = "d3")
    ])#914 for cut. 1036 for all

def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
autoencoder = Autoencoder(latent_dim)
#autoencoder.build(input_shape=np.shape(train_images))
epoch = 500

In [ ]:
adg = tf.keras.optimizers.Adagrad(
    learning_rate=0.01)

In [ ]:
adam = tf.keras.optimizers.Adam(
    learning_rate=0.0005)

In [ ]:
def loss1(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    d = tf.reduce_mean(tf.math.square(y_pred - y_true), axis=-1)
    #tf.print(d, [d], "Inside loss function")
    #tf.print(d.shape)
    return d

autoencoder.compile(optimizer=adam, loss=loss1)#es.MeanSquaredError())

try:
    os.mkdir(f"models/training{training_index}")
    checkpoint_path = f"models/training{training_index}/cp_{training_index}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
except:
    checkpoint_path = f"models/training{training_index}/cp_{training_index}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#autoencoder.summary()
#autoencoder.fit(train_images, train_images,
 #               epochs=10,
  #              shuffle=True,
   #             validation_data=(val_images,val_images))
hist = autoencoder.fit(train_cut, train_cut,
                epochs=epoch,
                shuffle=True,
                validation_data=(val_cut,val_cut),
                callbacks=[cp_callback])


In [ ]:
plt.figure()
x = np.linspace(1,epoch+1,epoch)
plt.plot(x,hist.history["loss"],label="loss")
plt.plot(x,hist.history["val_loss"],label="val loss")
plt.legend()
plt.savefig(f'Graphs/loss_{training_index}.png')
print(np.shape(train_images))

encoded_imgs = autoencoder.encoder(train_cut).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()
np.save(f"encoded_{training_index}",encoded_imgs)
np.save(f"decoded_{training_index}",decoded_imgs)

encoded_val = autoencoder.encoder(val_cut).numpy()
np.save(f"encoded_val_{training_index}",encoded_val)

In [ ]:
np.load("encoded_31_cut.npy")

In [ ]:
plt.figure()
plt.plot(train_cut[4])
plt.savefig("OG.png")
plt.figure()
plt.plot(decoded_imgs[4])
plt.savefig("RECON.png")
print(hist.history["val_loss"][-1])